In [1]:
import pandas as pd

#xls = pd.ExcelFile("LCDataDictionary.xlsx")
#df_dict_all = pd.concat([xls.parse(sheet) for sheet in xls.sheet_names], ignore_index=True)

In [139]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan

In [ ]:
spark = SparkSession.builder.getOrCreate()
df_loans = spark.read.csv("LCloan_raw_data.csv", header=True, inferSchema=True)

In [57]:
# שליפת סוגי העמודות
schema = df_loans.dtypes  # List of tuples: [('loan_amnt', 'double'), ('term', 'string'), ...]
schema

[('id', 'string'),
 ('member_id', 'string'),
 ('loan_amnt', 'int'),
 ('funded_amnt', 'int'),
 ('funded_amnt_inv', 'double'),
 ('term', 'string'),
 ('int_rate', 'double'),
 ('installment', 'double'),
 ('grade', 'string'),
 ('sub_grade', 'string'),
 ('emp_title', 'string'),
 ('emp_length', 'string'),
 ('home_ownership', 'string'),
 ('annual_inc', 'string'),
 ('verification_status', 'string'),
 ('issue_d', 'string'),
 ('loan_status', 'string'),
 ('pymnt_plan', 'string'),
 ('url', 'string'),
 ('desc', 'string'),
 ('purpose', 'string'),
 ('title', 'string'),
 ('zip_code', 'string'),
 ('addr_state', 'string'),
 ('dti', 'string'),
 ('delinq_2yrs', 'string'),
 ('earliest_cr_line', 'string'),
 ('inq_last_6mths', 'string'),
 ('mths_since_last_delinq', 'string'),
 ('mths_since_last_record', 'string'),
 ('open_acc', 'string'),
 ('pub_rec', 'string'),
 ('revol_bal', 'string'),
 ('revol_util', 'string'),
 ('total_acc', 'string'),
 ('initial_list_status', 'string'),
 ('out_prncp', 'string'),
 ('out_p

In [59]:
df1 = pd.read_csv("rejectedstat.csv")

In [61]:
df1

,RejectStats File,Description,column_name,source_sheet
0,Amount Requested,The total amount requested by the borrower,Amount Requested,RejectStats
1,Application Date,The date which the borrower applied,Application Date,RejectStats
2,Loan Title,The loan title provided by the borrower,Loan Title,RejectStats
3,Risk_Score,"For applications prior to November 5, 2013 the...",Risk_Score,RejectStats
4,Debt-To-Income Ratio,A ratio calculated using the borrower’s total ...,Debt-To-Income Ratio,RejectStats
5,Zip Code,The first 3 numbers of the zip code provided b...,Zip Code,RejectStats
6,State,The state provided by the borrower in the loan...,State,RejectStats
7,Employment Length,Employment length in years. Possible values ar...,Employment Length,RejectStats
8,Policy Code,publicly available policy_code=1\nnew products...,Policy Code,RejectStats


In [63]:
df2 = pd.read_csv("loanstatnew.csv")

In [65]:
df2

,LoanStatNew,Description,column_name,source_sheet
0,acc_now_delinq,The number of accounts on which the borrower i...,acc_now_delinq,RLoanStatNew
1,acc_open_past_24mths,Number of trades opened in past 24 months.,acc_open_past_24mths,RLoanStatNew
2,addr_state,The state provided by the borrower in the loan...,addr_state,RLoanStatNew
3,all_util,Balance to credit limit on all trades,all_util,RLoanStatNew
4,annual_inc,The self-reported annual income provided by th...,annual_inc,RLoanStatNew
...,...,...,...,...
146,settlement_status,The status of the borrower’s settlement plan. ...,settlement_status,RLoanStatNew
147,settlement_date,The date that the borrower agrees to the settl...,settlement_date,RLoanStatNew
148,settlement_amount,The loan amount that the borrower has agreed t...,settlement_amount,RLoanStatNew
149,settlement_percentage,The settlement amount as a percentage of the p...,settlement_percentage,RLoanStatNew


In [67]:
df3 = pd.read_csv("browsenotes.csv")

In [69]:
df3

,BrowseNotesFile,Description,column_name,source_sheet
0,acceptD,The date which the borrower accepted the offer,acceptD,BrowseNotes
1,accNowDelinq,The number of accounts on which the borrower i...,accNowDelinq,BrowseNotes
2,accOpenPast24Mths,Number of trades opened in past 24 months.,accOpenPast24Mths,BrowseNotes
3,addrState,The state provided by the borrower in the loan...,addrState,BrowseNotes
4,all_util,Balance to credit limit on all trades,all_util,BrowseNotes
...,...,...,...,...
115,sec_app_num_rev_accts,Number of revolving accounts at time of appli...,sec_app_num_rev_accts,BrowseNotes
116,sec_app_chargeoff_within_12_mths,Number of charge-offs within last 12 months a...,sec_app_chargeoff_within_12_mths,BrowseNotes
117,sec_app_collections_12_mths_ex_med,Number of collections within last 12 months e...,sec_app_collections_12_mths_ex_med,BrowseNotes
118,sec_app_mths_since_last_major_derog,Months since most recent 90-day or worse rati...,sec_app_mths_since_last_major_derog,BrowseNotes


### testing if there is a difference between column name to original sheet name column:

In [72]:
# לדוגמה עבור df1:
(df1['column_name'] != df1['RejectStats File']).sum()

0

In [74]:
(df2['column_name'] != df2['LoanStatNew']).sum()

0

In [76]:
(df3['column_name'] != df3['BrowseNotesFile']).sum()

0

In [79]:
# בחר רק את העמודות הרלוונטיות מכל אחד
df1_clean = df1[['column_name', 'Description', 'source_sheet']]
df2_clean = df2[['column_name', 'Description', 'source_sheet']]
df3_clean = df3[['column_name', 'Description', 'source_sheet']]

# איחוד לשליפה אחת
df_mapping = pd.concat([df1_clean, df2_clean, df3_clean], ignore_index=True)


In [81]:
df_mapping

,column_name,Description,source_sheet
0,Amount Requested,The total amount requested by the borrower,RejectStats
1,Application Date,The date which the borrower applied,RejectStats
2,Loan Title,The loan title provided by the borrower,RejectStats
3,Risk_Score,"For applications prior to November 5, 2013 the...",RejectStats
4,Debt-To-Income Ratio,A ratio calculated using the borrower’s total ...,RejectStats
...,...,...,...
275,sec_app_num_rev_accts,Number of revolving accounts at time of appli...,BrowseNotes
276,sec_app_chargeoff_within_12_mths,Number of charge-offs within last 12 months a...,BrowseNotes
277,sec_app_collections_12_mths_ex_med,Number of collections within last 12 months e...,BrowseNotes
278,sec_app_mths_since_last_major_derog,Months since most recent 90-day or worse rati...,BrowseNotes


In [87]:
df_schema = pd.DataFrame(schema, columns=["column_name", "column_type"])
df_schema

,column_name,column_type
0,id,string
1,member_id,string
2,loan_amnt,int
3,funded_amnt,int
4,funded_amnt_inv,double
...,...,...
140,settlement_status,string
141,settlement_date,string
142,settlement_amount,string
143,settlement_percentage,string


## Mapping DataType From Loans to each column in Dictionary

#### Normalization of the sheets dfs before merging

#### The names of column_name in df_mapping did not match to names in df_schema that were took from the df_loans

In [89]:
# ניקוי שם עמודה בשני הדאטהפריים
df_mapping['column_name'] = df_mapping['column_name'].str.strip().str.lower()
df_schema['column_name'] = df_schema['column_name'].str.strip().str.lower()

In [91]:
df_all = pd.merge(df_mapping, df_schema, on="column_name", how="left")

In [93]:
df_all['column_type'].isnull().sum()

67

In [95]:
missing_types = df_all[df_all['column_type'].isnull()]
print("columns without column types:", missing_types.shape[0])
print(missing_types[['column_name', 'source_sheet']])


columns without column types: 67
                 column_name source_sheet
0           amount requested  RejectStats
1           application date  RejectStats
2                 loan title  RejectStats
3                 risk_score  RejectStats
4       debt-to-income ratio  RejectStats
..                       ...          ...
261           totalbalexmort  BrowseNotes
262             totalbclimit  BrowseNotes
264    verified_status_joint  BrowseNotes
267   sec_app_fico_range_low  BrowseNotes
268  sec_app_fico_range_high  BrowseNotes

[67 rows x 2 columns]


In [97]:
print(missing_types['column_name'].unique())

['amount requested' 'application date' 'loan title' 'risk_score'
 'debt-to-income ratio' 'zip code' 'state' 'employment length'
 'policy code' 'fico_range_high' 'fico_range_low' 'last_fico_range_high'
 'last_fico_range_low' 'verified_status_joint' 'sec_app_fico_range_low'
 'sec_app_fico_range_high' 'acceptd' 'accnowdelinq' 'accopenpast24mths'
 'addrstate' 'annualinc' 'bcopentobuy' 'bcutil' 'creditpulld' 'delinq2yrs'
 'delinqamnt' 'earliestcrline' 'effective_int_rate' 'emplength' 'expd'
 'expdefaultrate' 'ficorangehigh' 'ficorangelow' 'fundedamnt'
 'homeownership' 'ils_exp_d' 'initialliststatus' 'inqlast6mths' 'intrate'
 'isincv' 'listd' 'loanamnt' 'memberid' 'mortacc' 'msa'
 'mths_since_oldest_il_open' 'mthssincelastdelinq' 'mthssincelastrecord'
 'mthssincemostrecentinq' 'mthssincerecentbc' 'mthssincerecentloandelinq'
 'mthssincerecentrevoldelinq' 'openacc' 'percentbcgt75' 'pubrec'
 'reviewstatus' 'reviewstatusd' 'revolbal' 'revolutil' 'servicefeerate'
 'subgrade' 'totalacc' 'totalbale

In [83]:
# pip install rapidfuzz

In [99]:
from rapidfuzz import process, fuzz

# רשימת שמות קיימים בקובץ ההלוואות
loan_columns = df_schema['column_name'].dropna().unique()
len(loan_columns)

145

In [103]:
missing_types = df_all[df_all['column_type'].isnull()].copy()
missing_types.count()

column_name     67
Description     67
source_sheet    67
column_type      0
dtype: int64

### Added type_status to map the missing column names types

In [107]:
df_all['type_status'] = df_all['column_type'].apply(lambda x: 'found' if pd.notnull(x) else 'missing')

In [147]:
df_all

,column_name,Description,source_sheet,type_status,column_name_fixed,column_type
0,amount requested,The total amount requested by the borrower,RejectStats,missing,amount requested,NaN
1,application date,The date which the borrower applied,RejectStats,missing,application date,NaN
2,loan title,The loan title provided by the borrower,RejectStats,missing,loan title,NaN
3,risk_score,"For applications prior to November 5, 2013 the...",RejectStats,missing,risk_score,NaN
4,debt-to-income ratio,A ratio calculated using the borrower’s total ...,RejectStats,missing,debt-to-income ratio,NaN
...,...,...,...,...,...,...
275,sec_app_num_rev_accts,Number of revolving accounts at time of appli...,BrowseNotes,found,sec_app_num_rev_accts,int
276,sec_app_chargeoff_within_12_mths,Number of charge-offs within last 12 months a...,BrowseNotes,found,sec_app_chargeoff_within_12_mths,int
277,sec_app_collections_12_mths_ex_med,Number of collections within last 12 months e...,BrowseNotes,found,sec_app_collections_12_mths_ex_med,int
278,sec_app_mths_since_last_major_derog,Months since most recent 90-day or worse rati...,BrowseNotes,found,sec_app_mths_since_last_major_derog,int


### Mergeing columns to find column types according to df_schema

In [113]:
from rapidfuzz import process, fuzz

# פונקציה שמקבלת שם עמודה (מ־df_mapping), ובודקת מה ההתאמה הכי קרובה אליו מ־existing_columns
def fuzzy_match_column(col_name, choices, threshold=90):
    match, score, _ = process.extractOne(col_name, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= threshold else None


In [123]:
df_missing = missing_types.copy()
existing_columns = df_schema['column_name'].unique()

In [125]:
df_missing['matched_column'] = df_missing['column_name'].apply(lambda x: fuzzy_match_column(x, existing_columns))

In [127]:
fix_map = df_missing.dropna(subset=['matched_column'])[['column_name', 'matched_column']].set_index('column_name').to_dict()['matched_column']

In [129]:
df_all['column_name_fixed'] = df_all['column_name'].replace(fix_map)

In [131]:
df_all = pd.merge(df_all.drop('column_type', axis=1),df_schema.rename(columns={'column_name': 'column_name_fixed'}),
                  on='column_name_fixed',
                  how='left')

In [149]:
df_all

,column_name,Description,source_sheet,type_status,column_name_fixed,column_type
0,amount requested,The total amount requested by the borrower,RejectStats,missing,amount requested,NaN
1,application date,The date which the borrower applied,RejectStats,missing,application date,NaN
2,loan title,The loan title provided by the borrower,RejectStats,missing,loan title,NaN
3,risk_score,"For applications prior to November 5, 2013 the...",RejectStats,missing,risk_score,NaN
4,debt-to-income ratio,A ratio calculated using the borrower’s total ...,RejectStats,missing,debt-to-income ratio,NaN
...,...,...,...,...,...,...
275,sec_app_num_rev_accts,Number of revolving accounts at time of appli...,BrowseNotes,found,sec_app_num_rev_accts,int
276,sec_app_chargeoff_within_12_mths,Number of charge-offs within last 12 months a...,BrowseNotes,found,sec_app_chargeoff_within_12_mths,int
277,sec_app_collections_12_mths_ex_med,Number of collections within last 12 months e...,BrowseNotes,found,sec_app_collections_12_mths_ex_med,int
278,sec_app_mths_since_last_major_derog,Months since most recent 90-day or worse rati...,BrowseNotes,found,sec_app_mths_since_last_major_derog,int


In [153]:
df_all.to_csv("mapping_updated.csv", index=False, header=True)

In [155]:
spark.stop()

### Mapping the relevant columns for testing -required

### Adding in the future:

df_merged["test_function"] = ""
df_merged["test_type"] = ""
df_merged["required"] = ""
df_merged["business_category"] = ""


In [61]:
def classify_column(description):
    d = str(description).lower()
    if 'income' in d or 'amount' in d or 'balance' in d:
        return 'funds'
    elif 'employer' in d or 'job' in d or 'title' in d:
        return 'customer details'
    elif 'status' in d or 'default' in d or 'delinquency' in d:
        return 'status'
    elif 'date' in d or 'month' in d:
        return 'dates'
    elif 'state' in d or 'zip' in d:
        return 'geographic location'
    else:
        'other'

df_merged['business_category'] = df_merged['Description'].apply(classify_column)
df_merged['business_Category']

df_merged.drop(columns=['business_Category'], inplace=True)


### Saving the file as main mapping file


df_merged.to_csv("test_mapping_master.csv", index=False, header=True)


df_merged.to_csv("test_mapping_master_clean.csv", index=False, encoding="utf-8", sep=",")


### mapping the testing functions 

### testing functions names:


['check_consistency', 'check_duplicates', 'check_foreign_key', 'check_nulls', 'count_records', 'dtype_consistency', 'duplicated_files_by_name_and_size', 'duplicated_files_current_path', 'file_timeliness', 'moving_average', 'validate_allowed_values', 'validate_date_format', 'validate_date_range', 'validate_numeric_range', 'validate_sum', 'validate_values_length']
